In [1]:
# Importar paqueteria necesaria
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [2]:
# Especificar ruta al repo clonado
os.chdir("/home/cesar/Documentos/Banco_Base")

# Lectura de la informacion
data_agosto = pd.read_csv('./Ejercicio_1/datasets/2024-08.csv')
data_sept = pd.read_csv('./Ejercicio_1/datasets/2024-09.csv')
data_oct = pd.read_csv('./Ejercicio_1/datasets/2024-10.csv')

data = pd.concat([data_agosto, data_sept, data_oct],ignore_index=True)
data.head(10)
 

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha_Arribo,Hora_Arribo
0,M,51.0,8620775,017,31/07/2024,23:53:06,061,01/08/2024,00:00:00
1,M,35.0,8882193,335,31/07/2024,23:46:43,320,01/08/2024,00:00:03
2,M,26.0,4755880,120,31/07/2024,23:50:45,111,01/08/2024,00:00:06
3,M,36.0,2983492,494,31/07/2024,23:20:42,199,01/08/2024,00:00:23
4,O,48.0,4972683,365,31/07/2024,22:39:20,348,01/08/2024,00:00:29
5,M,26.0,8515397,111,31/07/2024,23:55:52,018,01/08/2024,00:00:32
6,M,23.0,8018459,054,31/07/2024,23:41:25,484,01/08/2024,00:00:32
7,M,44.0,7868425,668,31/07/2024,23:44:54,654,01/08/2024,00:00:34
8,M,25.0,6066906,130,31/07/2024,23:51:57,158-159,01/08/2024,00:00:38
9,M,30.0,2091239,098,31/07/2024,23:56:58,096,01/08/2024,00:00:39


In [3]:
# Porcentaje de faltantes por columna
data['Genero_Usuario'] = data['Genero_Usuario'].replace('?', np.nan)
data.isna().mean() * 100


Genero_Usuario           0.010245
Edad_Usuario             0.004191
Bici                     0.000000
Ciclo_Estacion_Retiro    0.000000
Fecha_Retiro             0.000000
Hora_Retiro              0.000000
Ciclo_EstacionArribo     0.000000
Fecha_Arribo             0.000000
Hora_Arribo              0.000000
dtype: float64

In [ ]:
##### Inciso 1: Analisis exploratorio
# Numero de estaciones
print(pd.concat([data['Ciclo_Estacion_Retiro'], data['Ciclo_EstacionArribo']]).nunique())
# Histograma del genero de usuarios
frecuencias = data['Genero_Usuario'].value_counts(normalize=True).reset_index()
frecuencias.columns = ['Genero_Usuario', 'frecuencia']
plt.figure(figsize=(8, 6))
sns.barplot(x='Genero_Usuario', y='frecuencia', data=frecuencias, palette='pastel', edgecolor='black')
plt.title('Usuarios por genero')
plt.xlabel('Genero')
plt.ylabel('Frecuencia')
plt.ylim(0, 1)
plt.savefig('./Ejercicio_1/figures/genero_usuarios.png')
plt.show()

In [ ]:
# Cantidad de bicicletas usadas
print(data['Bici'].nunique())
# Cantidad de viajes por edad
plt.figure(figsize=(8, 6))
sns.histplot(data['Edad_Usuario'], bins=10, kde=True, color='blue')
plt.title('Edades')
plt.xlabel('Rangos de edad')
plt.ylabel('Cantidad de viajes')
plt.savefig('./Ejercicio_1/figures/edad_usuarios.png')
plt.show()
# Edad promedio por genero
print(data.groupby(['Genero_Usuario']).agg('Edad_Usuario').mean())

In [ ]:
# Calculo de duracion de viajes en minutos
data['Hora_Retiro'] = pd.to_datetime(data['Hora_Retiro'], format='%H:%M:%S')
data['Hora_Arribo'] = pd.to_datetime(data['Hora_Arribo'], format='%H:%M:%S')
data['duracion_viaje'] = (
    (data['Hora_Arribo'] - data['Hora_Retiro'])
    .dt.total_seconds()
    .div(60)    # Tiempo en minutos
    .mod(1440)  # Manejar diferencias de días
)
data[['Hora_Retiro','Hora_Arribo','duracion_viaje']].head(10)


In [ ]:
# Histograma de duraciones
plt.figure()
figsize=(8, 6)
sns.histplot(data['duracion_viaje'], kde=False, color='blue')
plt.title('Duracion en minutos')
plt.xlabel('Rangos de duracion')
plt.ylabel('Cantidad de viajes')
plt.xlim(0,100)
plt.savefig('./Ejercicio_1/figures/duracion_viajes.png')
plt.show()

In [21]:
############################## Inciso 2: Afluencia y outliers

(5798107, 10)